<a href="https://colab.research.google.com/github/luisosmx/API_connections/blob/main/API_ACROBAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Authorization Code

In [ ]:
import requests

url = "https://secure.echosign.com/public/oauth/v2"
redirect_uri = "https://oauth.pstmn.io/v1/callback"
client_id = ""
scopes = [
    "user_read:account",
    "user_write:account",
    "user_login:account",
    "agreement_read:account",
    "agreement_write:account",
    "agreement_send:account",
    "library_read:account",
    "library_write:account"
]

parameters = {
    "redirect_uri": redirect_uri,
    "response_type": "code",
    "client_id": client_id,
    "scope": " ".join(scopes)
}

response = requests.get(url, params=parameters)

print("Status code:", response.status_code)
print("Response URL:", response.url)





Status code: 200
Response URL: https://secure.na1.echosign.com/public/oauth/v2?redirect_uri=https%3A%2F%2Foauth.pstmn.io%2Fv1%2Fcallback&response_type=code&client_id=CBJCHBCAABAA5z5y6_YYaecwliRsbOJke0uGa4pet5Tr&scope=user_read%3Aaccount+user_write%3Aaccount+user_login%3Aaccount+agreement_read%3Aaccount+agreement_write%3Aaccount+agreement_send%3Aaccount+library_read%3Aaccount+library_write%3Aaccount


## Token request

In [ ]:

url = "https://secure.na1.echosign.com/oauth/v2/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Cache-Control": "no-cache"
}
data = {
    "grant_type": "authorization_code",
    "code": "CBNCKBAAHBCAABAAV6TWT01-xgIok6eKkxjVTImL4tKo-Vkq",
    "client_id": client_id,
    "client_secret": "7x2LZ8kCABcqdUTPgSFR5o01cVF1ED_k",
    "redirect_uri": redirect_uri
}

response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    token_data = response.json()
    access_token = token_data.get('access_token')
    print("Access Token:", access_token)
else:
    print("Failed to obtain access token. Status code:", response.status_code)
    print("Response:", response.text)


## Retrieves library documents for a user

In [ ]:
url_documentId = "https://api.na4.adobesign.com/api/rest/v6/libraryDocuments"

headers = {
    "Authorization": f"Bearer {access_token}"
}

response = requests.get(url_documentId, headers=headers)

if response.status_code == 200:
    print("Successful request")
    data = response.json()
    print(data)
else:

    print(f"Error: {response.status_code} - {response.text}")

## Send agreement

In [ ]:

url_post = "https://api.na4.adobesign.com/api/rest/v6/agreements"
libraryDocumentId = 'CBJCHBCAABAAtiJvomnfwQt_8LJWfy1-q4-sIR1bTOKY'
email = "luis.osmx16@gmail.com"

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

json_data = {
    "fileInfos": [{
         "libraryDocumentId": libraryDocumentId
    }],
    "name": "MyTestAgreement",
    "participantSetsInfo": [{
        "memberInfos": [{
            "email": email
        }],
        "order": 1,
        "role": "SIGNER"
    }],
    "signatureType": "ESIGN",
    "state": "IN_PROCESS"
}

response = requests.post(url_post, headers=headers, json=json_data)

if response.status_code == 201:
    print("Successful request")
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code} - {response.text}")


## Status of an agreement

In [ ]:
agreement_id = "CBJCHBCAABAATW_vUNo1daypYcYSIu2UGfQtnaW1-On8"


url_status = f'https://api.na4.adobesign.com:443/api/rest/v6/agreements/{agreement_id}'
params = {
    "agreementId": agreement_id
}

headers = {
    "Authorization": f"Bearer {access_token}"
}

response = requests.get(url_status, headers=headers)

if response.status_code == 200:
    print("Successful request")
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code} - {response.text}")


## Creates a reminder

In [ ]:
url_reminder = f'https://api.na4.adobesign.com:443/api/rest/v6/agreements/{agreement_id}/reminders'
recipientParticipantIds = 'CBJCHBCAABAAQ60d30bBfvCySUyJc7PUf2x2LEYN1oXp'
headers = {
    "Authorization": f"Bearer {access_token}"
}

json_data = {
  "recipientParticipantIds": [
    recipientParticipantIds
  ],
  "status": "ACTIVE"
}

response = requests.post(url_reminder, headers=headers,  json=json_data)

if response.status_code == 201:
    print("Successful request")
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code} - {response.text}")

## Retrieves the file stream


In [ ]:
# Retrieves the file stream

documentId = '3AAABLblqZhCiMfhf3neEK7iUYR1V827NzwSwUElGI57v74Lk9TBAs6M3kh0SAzqJu-nUj17BecRROLuhTlJf6HxJZUklRC7p'

url_document_retrives = f'https://api.na4.adobesign.com:443/api/rest/v6/agreements/{agreement_id}/documents/{documentId}'

headers = {
    "Authorization": f"Bearer {access_token}"
}

response = requests.get(url_document_retrives, headers=headers)

if response.status_code == 200:
    print("Successful request")
    with open("Document.pdf", "wb") as f:
        f.write(response.content)
    print("Document downloaded successfully")
else:
    print(f"Error: {response.status_code} - {response.text}")


## Class Adobe Acrobat API

In [ ]:
import requests

class AdobeAcrobatAPI:
    def __init__(self, client_id, client_secret, redirect_uri, scopes):
        self.client_id = client_id
        self.client_secret = client_secret
        self.redirect_uri = redirect_uri
        self.scopes = scopes
        self.access_token = None

    def get_authorization_code(self):
        url = "https://secure.echosign.com/public/oauth/v2"

        parameters = {
            "redirect_uri": self.redirect_uri,
            "response_type": "code",
            "client_id": self.client_id,
            "scope": " ".join(self.scopes)
        }

        response = requests.get(url, params=parameters)

        if response.status_code == 200:
            return response.url
        else:
            print("Failed to obtain authorization code.")
            return None

    def get_access_token(self, code):
        url = "https://secure.na1.echosign.com/oauth/v2/token"

        headers = {
            "Content-Type": "application/x-www-form-urlencoded",
            "Cache-Control": "no-cache"
        }
        data = {
            "grant_type": "authorization_code",
            "code": code,
            "client_id": self.client_id,
            "client_secret": self.client_secret,
            "redirect_uri": self.redirect_uri
        }

        response = requests.post(url, headers=headers, data=data)

        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data.get('access_token')
            return self.access_token
        else:
            print("Failed to obtain access token.", response.status_code)
            print("Response:", response.text)
            return None

    def retrieve_library_documents(self):
        url_documentId = "https://api.na4.adobesign.com/api/rest/v6/libraryDocuments"

        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }

        response = requests.get(url_documentId, headers=headers)

        if response.status_code == 200:
            print("Successful request")
            data = response.json()
            return data
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None

    def send_agreement(self, library_document_id, email):
        url_post = "https://api.na4.adobesign.com/api/rest/v6/agreements"

        headers = {
            "Authorization": f"Bearer {self.access_token}",
            "Content-Type": "application/json"
        }

        json_data = {
            "fileInfos": [{
                 "libraryDocumentId": library_document_id
            }],
            "name": "MyTestAgreement",
            "participantSetsInfo": [{
                "memberInfos": [{
                    "email": email
                }],
                "order": 1,
                "role": "SIGNER"
            }],
            "signatureType": "ESIGN",
            "state": "IN_PROCESS"
        }

        response = requests.post(url_post, headers=headers, json=json_data)

        if response.status_code == 201:
            print("Successful request")
            data = response.json()
            return data
        else:
            print(f"Error: {response.status_code} - {response.text}")
            return None

    def status_agreement(self, agreement_id):
      url_status = f'https://api.na4.adobesign.com:443/api/rest/v6/agreements/{agreement_id}'

      headers = {
          "Authorization": f"Bearer {self.access_token}"
      }

      response = requests.get(url_status, headers=headers)

      if response.status_code == 200:
          print("Successful request")
          data = response.json()
          return data
      else:
          print(f"Error: {response.status_code} - {response.text}")
          return None

    def create_reminder(self, recipientParticipantIds):
      url_reminder = f'https://api.na4.adobesign.com:443/api/rest/v6/agreements/{self.agreement_id}/reminders'

      headers = {
          "Authorization": f"Bearer {self.access_token}"
      }

      json_data = {
        "recipientParticipantIds": [
          recipientParticipantIds
        ],
        "status": "ACTIVE"
      }

      response = requests.post(url_reminder, headers=headers,  json=json_data)

      if response.status_code == 201:
          print("Successful request")
          data = response.json()
          return data
      else:
          print(f"Error: {response.status_code} - {response.text}")
          return None

    def retrieves_id(self, agreement_id):

      url_document = f'https://api.na4.adobesign.com:443/api/rest/v6/agreements/{agreement_id}/documents'

      headers = {
          "Authorization": f"Bearer {self.access_token}"
      }

      response = requests.get(url_document, headers=headers)

      if response.status_code == 200:
          print("Successful request")
          data = response.json()
          return data
      else:
          print(f"Error: {response.status_code} - {response.text}")
          return None

    def retrives_file_stream(self, documentId):
      url_document_retrives = f'https://api.na4.adobesign.com:443/api/rest/v6/agreements/{agreement_id}/documents/{documentId}'

      headers = {
          "Authorization": f"Bearer {self.access_token}"
      }

      response = requests.get(url_document_retrives, headers=headers)

      if response.status_code == 200:
          print("Successful request")
          with open("Document.pdf", "wb") as f:
              f.write(response.content)
          print("Document downloaded successfully")
      else:
          print(f"Error: {response.status_code} - {response.text}")
          return None



client_id = ""
client_secret = ""
redirect_uri = "https://oauth.pstmn.io/v1/callback"
scopes = [
    "user_read:account",
    "user_write:account",
    "user_login:account",
    "agreement_read:account",
    "agreement_write:account",
    "agreement_send:account",
    "library_read:account",
    "library_write:account"
]
recipientParticipantIds = 'CBJCHBCAABAAQ60d30bBfvCySUyJc7PUf2x2LEYN1oXp'
documentId = '3AAABLblqZhCiMfhf3neEK7iUYR1V827NzwSwUElGI57v74Lk9TBAs6M3kh0SAzqJu-nUj17BecRROLuhTlJf6HxJZUklRC7p'
acrobat_api = AdobeAcrobatAPI(client_id, client_secret, redirect_uri, scopes)


In [ ]:
code = acrobat_api.get_authorization_code()
print(code)

In [ ]:
code = 'CBNCKBAAHBCAABAAwfaac96LDZiGgNHiCtIEK8fdTbyh5ee-'
token = acrobat_api.get_access_token(code)
print(token)

In [ ]:
library = acrobat_api.retrieve_library_documents()
print(library)

In [ ]:
libraryDocumentId = 'CBJCHBCAABAAdtJ9e85aEBtlWweweXUi_IWbq4j1h0pb'
email = 'luis.osmx16@gmail.com'
send = acrobat_api.send_agreement(libraryDocumentId, email)
print(send)


In [ ]:
agreement_id = "CBJCHBCAABAAQCQbcRrHtbW_2PMqeybiX1Y-e4uyLrtd"

status = acrobat_api.status_agreement(agreement_id)
print(status)

In [ ]:
agreement_id = "CBJCHBCAABAAQCQbcRrHtbW_2PMqeybiX1Y-e4uyLrtd"
retrivefile = acrobat_api.retrieves_id(agreement_id)
print(retrivefile)

In [ ]:
id_document = '3AAABLblqZhD_3JmyN4D3_doLqpYz22D3FNNvxiZJP1CNMf7iWWDJnENSEn-2nbtlw3pfHMZrM3gEwkJ4fkhnIBxhHHxFHP6z'
document = acrobat_api.retrives_file_stream(id_document)